# Decorators

**Agenda**

 * What are decorators
 * Some examples:
   * lru_cache
   * Properties
   * Just in time compilation

### A quick recap on higher order functions

We can create a function that defines a function and returns it

In [56]:
def add(x, y):
    return x + y


# Students make this
def add_curried(x):
    def add_x(y):
        return x + y
    return add_x

add_3 = add_curried(3)
print(add_3(5))

8


We can also create a function that takes a function as input

In [55]:
def apply_to_list_of_lists(f, list_of_lists):
    return [[f(l) for l in list] for list in list_of_lists]

list_of_lists = [[3, 2], [5], [6, 7]]
print(apply_to_list_of_lists(add_3, list_of_lists))

[[6, 5], [8], [9, 10]]


## A quick interlude: lambda functions
Expressions you will see, but should rarely use

In [3]:
apply_to_list_of_lists(lambda x: x - 2, list_of_lists)

[[1, 0], [3], [4, 5]]

A lambda function is a function that you define in one line, akin to anonymous functions in MATLAB.

In [4]:
subtract_2 = lambda x: x - 2
print(subtract_2(3))

1


You should never store a lambda function in a variable (like we just did), instead, you should define a one-line function.
The only time you use lambda functions are if you have a function that takes a function as input. Here is an example:

In [57]:
numbers = [3, -5, -1, 0, 2, 1]

print(sorted(numbers, key=lambda x: (x-2)**2))

[2, 3, 1, 0, -1, -5]


## Tuple- and dictionary unpacking
An essential part of functional programming in Python is tuple unpacking and dictionary unpacking.

**Tuple unpacking**

In [6]:
def f(x, *args):
    print(args)

In [7]:
f(3)

()


In [8]:
f(3, 4, 5)

(4, 5)


In [9]:
f()

TypeError: f() missing 1 required positional argument: 'x'

We see here that the function `f` takes one argument, x, and any "extra" arguments are put in the args tuple.

In [10]:
def f(x, y=3, *extra_args):
    print(extra_args)

In [11]:
f(3, 2)

()


In [12]:
f(3, 2, 5, 6)

(5, 6)


In [13]:
f(3, y=2, 5)

SyntaxError: positional argument follows keyword argument (<ipython-input-13-adc44a4ecd27>, line 1)

We can also use a tuple to pass arguments to a function

In [14]:
args = (3, 5, 6)
f(*args)

(6,)


In [15]:
f(3, 5, 6)

(6,)


We see that the `*` before a tuple "unpacks" the contents when used in a function call

**Dictionary unpacking**

In [16]:
def f(x, **kwargs):
    print(kwargs)


In [17]:
f(3, 2)

TypeError: f() takes 1 positional argument but 2 were given

In [18]:
f(3, y=2, z=3)

{'y': 2, 'z': 3}


We see that the *extra* keyword arguments are put in the kwargs dictionary

In [19]:
def f(x, *extra_args, **extra_kwargs):
    print('x: ', x)
    print('extra_args: ', extra_args)
    print('extra_kwargs: ', extra_kwargs)

In [20]:
f(3, 2, 1, y=9)

x:  3
extra_args:  (2, 1)
extra_kwargs:  {'y': 9}


We can also use dictionary unpacking when we call a function

In [21]:
kwargs = {'x': 5}
f(**kwargs)

x:  5
extra_args:  ()
extra_kwargs:  {}


### When can this be useful?

When we create functions and we don't know which inputs it will take.


In [22]:
def print_message(f):
    def new_f(*args, **kwargs):
        print('Hello!')
        return f(*args, **kwargs)
    return new_f

In [23]:
add_3_with_message = print_message(add_3)

What will happen here?

In [24]:
add_3_with_message(4)

Hello!


7

Let us modify this so we can add our own message

In [25]:
def print_custom_message(message, f):
    def new_f(*args, **kwargs):
        print(message)
        return f(*args, **kwargs)
    return new_f

In [26]:
add_3_with_message = print_custom_message('Nice to meet you!', add_3)

In [27]:
add_3_with_message(5)

Nice to meet you!


8

# Decorators
A decorator is a function that takes a function as its only input and returns a new function

In [28]:
@print_message
def add_2(x):
    return x + 2

In [29]:
add_2(5)

Hello!


7

This is equivalent to writing

In [30]:
def add_2(x):
    return x + 2
add_2 = print_message(add_2)

So why use decorators? The code is easier to read. We decorate the functions we define with some tags that modifies their behaviour.

**Can we use the `print_custom_message` function as a decorator?**

Not as it currently stands, however if we use currying to separate the function argument from the rest, then we can.

In [31]:
def print_custom_message(message):
    def decorator(f):
        def new_f(*args, **kwargs):
            print(message)
            return f(*args, **kwargs)
        return new_f
    return decorator

In [32]:
@print_custom_message('Decorators are cool')
def subtract_2(x):
    return x - 2

subtract_2(5)

Decorators are cool


3

## Useful decorators
One way to use decorators is to cache the inputs and outputs of a function

In [33]:
def cache_function(f):
    cache = {}
    def cached_function(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    return cached_function

In [34]:
from time import sleep

@cache_function
def slow_add_3(x):
    sleep(5)
    return x + 3

In [35]:
print(slow_add_3(5))
print(slow_add_3(6))

8
9


In [36]:
print(slow_add_3(5))
print(slow_add_3(6))

8
9


Better alternative: `lru_cache`

In [37]:
from functools import lru_cache

@lru_cache()  # The decorator takes optional keyword arguments as input
def slow_add_2(x):
    sleep(2)
    return x + 2

In [38]:
print(slow_add_2(5))
print(slow_add_2(6))

7
8


In [39]:
print(slow_add_2(5))
print(slow_add_2(6))

7
8


## Another example: Properties

This is useful for the exam!

A benefit of object oriented programming is to hide the implementation from the interface. To do this, we often use *hidden variables*, that is variables that are unavailable "outside" the class. In Python, we don't have such variables. Instead, we name variables that shouldn't be used outside the class with a SINGLE leading underscore.

In [40]:
class Vector:
    def __init__(self, position):
        self._position = position
        self.num_position_reads = 0
    
    def get_position(self):
        self.num_position_reads += 1
        return self._position

In [41]:
v = Vector([1, 2])

In [42]:
print(v.get_position())
print(v.num_position_reads)
print(v.get_position())
print(v.num_position_reads)

[1, 2]
1
[1, 2]
2


This is very unpythonic. We use properties instead of getters.

In [43]:
class Vector:
    def __init__(self, position):
        self._position = position
        self.num_position_reads = 0
    
    @property
    def position(self):
        self.num_position_reads += 1
        return self._position

In [44]:
v = Vector([1, 2])

In [45]:
print(v.position)
print(v.num_position_reads)
print(v.position)
print(v.num_position_reads)

[1, 2]
1
[1, 2]
2


The position attribute acts like a variable, but is instead a property. This is a very useful feature in Python.

In [46]:
class Vector:
    def __init__(self, position):
        self._position = position
        self.num_position_reads = 0
        self.num_position_writes = 0
    
    @property
    def position(self):
        self.num_position_reads += 1
        return self._position
    
    @position.setter
    def position(self, value):
        self.num_position_writes +=1
        self._position = value

In [47]:
v = Vector([1, 2])
v.position = [5, 2]

In [48]:
v.num_position_writes

1

In [49]:
v.position
v.num_position_reads

1

Properties are very useful, as they allow us to use getters and setters without the ugly syntax often seen in other languages, but they can be misleading, as we normaly think that accessing an attribute takes a short time, which it may not in this case.

## Final example: Just In Time (JIT) compilation

In [50]:
def sum_n_squares(n):
    sum = 0
    for i in range(n):
        sum += i**2
    return sum

In [51]:
%timeit sum_n_squares(10000)

2.87 ms ± 93.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
from numba import jit

@jit()
def sum_n_squares_fast(n):
    sum = 0
    for i in range(n):
        sum += i**2
    return sum

In [53]:
sum_n_squares_fast(10000)  # First call takes longer
%timeit sum_n_squares_fast(10000)

198 ns ± 4.19 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


x10 000 speedup!

# Class exersices

1. Create a decorator, `print_messages`, similar to `print_message`, but it should print `"Calling function"` before calling the decorated function and `"Finished calling function"` after calling the decorated function.
2. Create a decorator, `print_custom_messages`, similar to `print_custom_message`, but it should take two messages as input, a message to print before calling the decorated function and a message to print after.
3. Extend the following code
```python
    class Square:
        def __init__(self, top_left, line_length):
            x, y = top_left
            self.corners = [
                (x, y),
                (x + line_length, y),
                (x + line_length, y + line_length),
                (x, y + line_length)
            ]
```
so that it has a property `centre` that computes and returns the centre of the square.